In [53]:
#import all packages
import pandas as pd
import numpy as np
import re as re
import nltk
from nltk.stem.porter import *

from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import nltk.classify.util
from nltk.classify import NaiveBayesClassifier

from datetime import datetime, timedelta

In [54]:
def reduce_lengthening(text):
    pattern = re.compile(r"(.)\1{2,}")
    return pattern.sub(r"\1\1", text)

In [55]:
# datafile
data_file = "data_with_topic_v2.csv"
data = pd.read_csv(data_file)
data.columns = data.columns.str.strip().str.replace(" ","_") # reason for doing this is cause cannot get columns with space into a list

#reformat the respective columns

#lower case
data['Review'] = [word.lower() for word in data['Review']]

#Strip off the Variation Word
data['Product_Purchase'] = data['Product_Purchase'].str.strip("Variation:")

#Change Date of Review to DateTime
#data['Date_Of_Review'] = pd.to_datetime(data['Date_Of_Review'])

data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8987 entries, 0 to 8986
Data columns (total 14 columns):
Unnamed:_0          8987 non-null int64
Unnamed:_0.1        8987 non-null int64
Platform            8987 non-null object
Brand               8987 non-null object
Category            8987 non-null object
Product_Name        8987 non-null object
Price               8987 non-null float64
Reviewer            8965 non-null object
Review              8987 non-null object
Product_Purchase    7382 non-null object
Ratings             8965 non-null float64
Date_Of_Review      8965 non-null object
Response            8965 non-null object
topic               8987 non-null object
dtypes: float64(2), int64(2), object(10)
memory usage: 983.0+ KB


,Unnamed:_0,Unnamed:_0.1,Platform,Brand,Category,Product_Name,Price,Reviewer,Review,Product_Purchase,Ratings,Date_Of_Review,Response,topic
0,0,0,Shopee,L'Oreal Paris,Serum & Treatment,L'Oreal Paris Revitalift Crystal Micro-Essence...,25.9,zenheng,"delivery was quite fast, and item was on disco...",NaN,4.0,2/6/2020 23:58,no,delivery
1,1,1,Shopee,L'Oreal Paris,Serum & Treatment,L'Oreal Paris Revitalift Crystal Micro-Essence...,25.9,glennoeestaquio,received item sealed properly.\r\r\r\r\nquick ...,NaN,5.0,2/6/2020 12:44,no,delivery
2,2,2,Shopee,L'Oreal Paris,Serum & Treatment,L'Oreal Paris Revitalift Crystal Micro-Essence...,25.9,shy2206,yet to try but delivery was prompt and product...,NaN,5.0,2/6/2020 10:21,no,delivery
3,3,3,Shopee,L'Oreal Paris,Serum & Treatment,L'Oreal Paris Revitalift Crystal Micro-Essence...,25.9,lindamiyalin,"well wrapped, in good condition, super fast ...",NaN,5.0,1/30/2020 23:01,no,delivery
4,4,4,Shopee,L'Oreal Paris,Serum & Treatment,L'Oreal Paris Revitalift Crystal Micro-Essence...,25.9,kyc385,"well packed, will buy again",NaN,5.0,2/1/2020 11:40,no,delivery


### Transformation and Initalise of data

In [56]:
# make the old csv data into a list

row_id_list = data['Unnamed:_0'].tolist()
platform_list = data['Platform'].tolist()
brand_list = data['Brand'].tolist()
category_list = data['Category'].tolist()
product_name_list = data['Product_Name'].tolist()
price_list = data['Price'].tolist()
reviewer_list = data['Reviewer'].tolist()
review_list = data['Review'].tolist()
product_variation_list = data['Product_Purchase'].tolist()
rating_list = data['Ratings'].tolist()
date_review_list = data["Date_Of_Review"].tolist()
response_list = data["Response"].tolist()
topic_list = data["topic"].tolist()

# initialising the new columns

id_csv=[]
platform_csv=[]
brand_csv = []
category_csv = []
product_name_csv = []
prices_csv = []
reviewer_csv =[]
review_splitted_csv = []
review_csv=[]
product_variation_csv = []
rating_csv = []
date_review_csv = []
response_csv = []
topic_csv=[]

# this is the length of the csv old data
print ("The length of the dataset is:", len(brand_list))

The length of the dataset is: 8987


In [57]:
#upload the appos
from word_dict import appos

print(appos)

{"aren't": 'are not', "can't": 'cannot', "couldn't": 'could not', "didn't": 'did not', "doesn't": 'does not', "don't": 'do not', "hadn't": 'had not', "hasn't": 'has not', "haven't": 'have not', "he'd": 'he would', "he'll": 'he will', "he's": 'he is', "i'd": 'I had', "i'll": 'I will', "i'm": 'I am', "isn't": 'is not', "it's": 'it is', "it'll": 'it will', "i've": 'I have', "let's": 'let us', "l'oreal": 'loreal', "mightn't": 'might not', "mustn't": 'must not', "shan't": 'shall not', "she'd": 'she would', "she'll": 'she will', "she's": 'she is', "shouldn't": 'should not', "that's": 'that is', "there's": 'there is', "they'd": 'they would', "they'll": 'they will', "they're": 'they are', "they've": 'they have', "we'd": 'we would', "we're": 'we are', "weren't": 'were not', "we've": 'we have', "what'll": 'what will', "what're": 'what are', "what's": 'what is', "what've": 'what have', "where's": 'where is', "who'd": 'who would', "who'll": 'who will', "who're": 'who are', "who's": 'who is', "who'

### Transform the Date Format [Optional]

In [58]:
new_date =[]
for i in range(len(brand_list)):
    
    date_review = date_review_list[i]
    
    if platform_list[i] == 'Lazada':
        if 'days ago' in date_review or 'day ago' in date_review :
            day = date_review.replace('days ago', '').replace('day ago', '')
            date_review = datetime.today() - timedelta(days=int(day))
            date_review = date_review.strftime("%d-%b-%y")
#             print("trans",date_review)

        elif 'weeks ago' in date_review or 'week ago' in date_review:
            day = date_review.replace('weeks ago', '').replace('week ago', '')
            day = int(day)*7
            date_review = datetime.today() - timedelta(days=day)
            date_review = date_review.strftime("%d-%b-%y")
#             print("trans",date_review)

        elif 'hours ago' in date_review or 'hour ago' in date_review:
            hours = date_review.replace('hours ago', '').replace('hour ago', '')
            date_review = datetime.today() - timedelta(hours=int(hours))
            date_review = date_review.strftime("%d-%b-%y")
    
#         print("trans",date_review)
        new_date.append(date_review)
    else:
#         print(platform_list[i])
        new_date.append(date_review)
        
data["Date_Of_Review"] = new_date

new_date_review_list = data["Date_Of_Review"].tolist()

### Spilt the Sentence 

In [59]:
import itertools
from spellchecker import SpellChecker
from nltk.tokenize import word_tokenize

spell = SpellChecker()

for i in range(len(brand_list)):
    # current_review is a string
    current_review = review_list[i]
    
    #spilt the sentence
    splitted_sentence= re.split(r'[.!]', current_review) 
    #print(splitted_sentence)
    
    for j in range(len(splitted_sentence)):
        
        splitted_sentence[j].strip()
        if len(splitted_sentence[j]) > 0 :
            
            processed_review = splitted_sentence[j].strip()
            
            # Negation handling
            processed_review = processed_review.split()
            processed_review =[appos[w] if w in appos else w for w in processed_review]
            processed_review = " ".join(processed_review) 
            
            processed_review = processed_review.strip()
            # Remove all the special characters
            processed_review = re.sub(r'\W', ' ', processed_review)
            # Removing prefixed 'b'
            processed_review = re.sub(r'^b\s+', '', processed_review)
            # remove all single characters contains a white space character
            processed_review= re.sub(r'^[a-zA-Z]$', ' ', processed_review)
            # Substituting multiple spaces with single space
            processed_review = re.sub(r'\s+', ' ', processed_review, flags=re.I)
          
            
            processed = processed_review.strip()
            #print(processed)
            
            # final check if the processed_review is empty
            final = []
            if processed != "":
                tokenized = word_tokenize(processed)
                #removed additional letters eg "gooooooodddddd" to "good"
                for w in tokenized:
                    processed_2 = reduce_lengthening(w)
                    processed_2 = spell.correction(processed_2)
                    final.append(processed_2)
                final =' '.join(final)
                #print(final)
                # append into a dictionary
                platform_csv.append(platform_list[i])
                id_csv.append(row_id_list[i])
                brand_csv.append(brand_list[i])
                category_csv.append(category_list[i])
                product_name_csv.append(product_name_list[i])
                prices_csv.append(price_list[i])
                reviewer_csv.append(reviewer_list[i])
                review_csv.append(current_review)
                review_splitted_csv.append(final)
                product_variation_csv.append(product_variation_list[i])
                rating_csv.append(rating_list[i])
                date_review_csv.append(new_date_review_list[i])
                response_csv.append(response_list[i])
                topic_csv.append(topic_list[i])


In [60]:
#Store into a new data frame
splitted_data = {'ID':id_csv,'Platform':platform_csv ,'Brand':brand_csv, 'Category': category_csv, 'Product Name ': product_name_csv, 'Price':prices_csv ,'Reviewer':reviewer_csv,'Review':review_csv, 'Review_splitted':review_splitted_csv, 'Product Purchase':product_variation_csv,'Ratings':rating_csv,'Date Of Review':date_review_csv,'Response': response_csv , 'Topic':topic_csv}
splitted_df = pd.DataFrame.from_dict(splitted_data)
splitted_df['Date Of Review'] = pd.to_datetime(splitted_df['Date Of Review'])
splitted_df.head()
splitted_df.info()

splitted_df.to_csv('AllData_Process.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17119 entries, 0 to 17118
Data columns (total 14 columns):
ID                  17119 non-null int64
Platform            17119 non-null object
Brand               17119 non-null object
Category            17119 non-null object
Product Name        17119 non-null object
Price               17119 non-null float64
Reviewer            17097 non-null object
Review              17119 non-null object
Review_splitted     17119 non-null object
Product Purchase    14421 non-null object
Ratings             17097 non-null float64
Date Of Review      17097 non-null datetime64[ns]
Response            17097 non-null object
Topic               17119 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(10)
memory usage: 1.8+ MB


### Create the Training Data Set

In [61]:
# spacy for lemmatization
import spacy

def lemmatization(texts):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = sp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc ])
    return texts_out

In [62]:
from nltk.corpus import stopwords
from nltk.corpus import PlaintextCorpusReader


review_cleaned=[]
stop_list = stopwords.words('english')

stemmer = PorterStemmer()
sp = spacy.load('en', disable=['parser', 'ner'])

splitted_df= splitted_df[splitted_df.Review_splitted != 'no comments review is an image']
tokenized_review = splitted_df["Review_splitted"]


for sentence in tokenized_review:
    tokenized_review = nltk.word_tokenize(sentence)
    review_stopremoved = [w for w in tokenized_review if w not in stop_list]
#     review_translate = [gs.translate(w,'en') for w in review_stopremoved if re.findall(r'[\u4e00-\u9fff]+', w)]
#     print(review_translate)
    review_cleaned.append(review_stopremoved)

# Lemmatized all the words
data_lemmatized = lemmatization(review_cleaned)
#print(data_lemmatized)


#splitted_df['tokenized_review'] = data_lemmatized    
splitted_df.head()

,ID,Platform,Brand,Category,Product Name,Price,Reviewer,Review,Review_splitted,Product Purchase,Ratings,Date Of Review,Response,Topic
0,0,Shopee,L'Oreal Paris,Serum & Treatment,L'Oreal Paris Revitalift Crystal Micro-Essence...,25.9,zenheng,"delivery was quite fast, and item was on disco...",delivery was quite fast and item was on discou...,NaN,4.0,2020-02-06 23:58:00,no,delivery
1,0,Shopee,L'Oreal Paris,Serum & Treatment,L'Oreal Paris Revitalift Crystal Micro-Essence...,25.9,zenheng,"delivery was quite fast, and item was on disco...",so far liking this product,NaN,4.0,2020-02-06 23:58:00,no,delivery
2,0,Shopee,L'Oreal Paris,Serum & Treatment,L'Oreal Paris Revitalift Crystal Micro-Essence...,25.9,zenheng,"delivery was quite fast, and item was on disco...",will continue to use it,NaN,4.0,2020-02-06 23:58:00,no,delivery
3,1,Shopee,L'Oreal Paris,Serum & Treatment,L'Oreal Paris Revitalift Crystal Micro-Essence...,25.9,glennoeestaquio,received item sealed properly.\r\r\r\r\nquick ...,received item sealed properly,NaN,5.0,2020-02-06 12:44:00,no,delivery
4,1,Shopee,L'Oreal Paris,Serum & Treatment,L'Oreal Paris Revitalift Crystal Micro-Essence...,25.9,glennoeestaquio,received item sealed properly.\r\r\r\r\nquick ...,quick delivery too,NaN,5.0,2020-02-06 12:44:00,no,delivery


In [63]:
# ## translator
# import goslate

# text = "价格合理"

# gs = goslate.Goslate()
# translatedText = gs.translate(text,'en')
# print(translatedText)

### Import the Positive , Negative , Increment , Decrement and Inverse Words


In [64]:
import nltk 

pos_lexicon = 'dict/positive-words.txt'
neg_lexicon = 'dict/negative-words.txt'
inc_words = 'dict/increment-words.txt'
dec_words = 'dict/decrement-words.txt'
inv_words = 'dict/inverse-words.txt'


# Read the positive sentiment lexicon.
pos_dict = {}
f = open(pos_lexicon, 'r', encoding = "ISO-8859-1")
for line in f:
    line = line.strip()
    pos_dict[line] = 1
f.close()

# Read the negative sentiment lexicon.
neg_dict = {}
f = open(neg_lexicon, 'r', encoding = "ISO-8859-1")
for line in f:
    line = line.strip()
    neg_dict[line] = 1
f.close()


# Read the increment words.
inc_dict = {}
f = open(inc_words, 'r', encoding = "ISO-8859-1")
for line in f:
    line = line.strip()
    inc_dict[line] = 1
f.close()

# Read the decrement words.
dec_dict = {}
f = open(dec_words, 'r', encoding = "ISO-8859-1")
for line in f:
    line = line.strip()
    dec_dict[line] = 1
f.close()

# Read the inverse words.
inv_dict = {}
f = open(inv_words, 'r', encoding = "ISO-8859-1")
for line in f:
    line = line.strip()
    inv_dict[line] = 1
f.close()


### Lexicon-based Sentiment Polarity Classification

In [65]:
predicted_labels = []
score_label=[]

for sent in data_lemmatized:
    total_score = 0
   # print(sent)
    for w in sent:
        score = 0
        if w in pos_dict:
            score = 1
        elif w in neg_dict:
            score = -1
        #check previous word
        position = sent.index(w)
        previous_word = sent[position-1]
        if previous_word is not None:
            if previous_word in inc_dict : 
                score *= 2.0
            elif previous_word in dec_dict:
                score /=2
            elif previous_word in inv_dict:
                score *= -1  
        total_score+=score
    
    if total_score >0:
        predicted_labels.append('1')
        
    elif total_score <0:
        predicted_labels.append('-1')
    else:
        predicted_labels.append('0')
        
    score_label.append(int(total_score))
    #print(total_score)

splitted_df["Polarity"] = predicted_labels
splitted_df["Score"] = score_label
# splitted_df.to_csv('AllData_Sentiment.csv')


### Combine Polarity for Splitted Review

In [66]:
from statistics import mode

#find the mode of each review
result = splitted_df.groupby('ID')['Polarity'].agg(lambda x:x.value_counts().index[0])
#print(result)

# insert a overall polarity col, with the mode value
splitted_df['Overall Polarity'] = [result[id] for id in splitted_df['ID']]
splitted_df.to_csv('AllData_DashBoard.csv')